In [1]:
# Import library-library
import os
import glob

# Data Preparation and Preprocessing
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
NLTK_StopWords = stopwords.words('indonesian')
NLTK_StopWords = set(NLTK_StopWords)

# Word Embedding
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

from sklearn.metrics.pairwise import cosine_similarity

# Input and Expansion Query
from textblob import TextBlob
from nltk.tokenize import wordpunct_tokenize
#from googletrans import Translator

#Split to train and test
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
excluded_words = ["tempat", "waktu"]
NLTK_StopWords.update(["detik", "detikjatim", "detikjateng", "detikjabar", "detiksulsel", "detiksumbar", "detikbali", "detikpapua", "detiksulteng", "detikmaluku", "detjatim", "detikcom"])

In [3]:
def preprocessing(berita):
    s = berita.lower()
    s = s.replace('\n', ' ')
    s = s.replace('\r', ' ')
    s = s.replace(' o ', ' ')
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    s = re.sub(r'[0-9]', ' ', s)
    tokens = [token for token in s.split(" ") if token != ""]
    T = [t for t in tokens if ((t in excluded_words) or (t not in NLTK_StopWords))]
    return T

In [5]:
df =pd.read_csv("scrapped_news.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1804 entries, 0 to 1803
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1804 non-null   object
 1   date         1804 non-null   object
 2   description  1804 non-null   object
 3   source       1804 non-null   object
dtypes: object(4)
memory usage: 56.5+ KB


,title,date,description,source
0,"Usai Kampung Bahari, Kini Kampung Boncos 'Diob...","Kamis, 10 Mar 2022 17:43 WIB\n\n",Penindakan terhadap peredaran\n\n Kapolsek Pal...,www.detik.com
1,5 Fakta Razia Kampung Bahari Berujung Terkuak ...,"Kamis, 10 Mar 2022 08:04 WIB\n\n",", Tanjung Priok, Jakarta Utara, kembali digere...",www.detik.com
2,Bandar Narkoba di Kampung Bahari Sembunyikan S...,"Rabu, 09 Mar 2022 20:59 WIB\n\n",Polisi menyita 22 unit motor dalam penggerebek...,www.detik.com
3,Kampung Narkoba di Deli Serdang Sumut Digerebe...,"Kamis, 10 Mar 2022 04:30 WIB\n\n",Polisi menggerebek\n\n Penggerebekan itu dilak...,www.detik.com
4,"Kode Bandar Narkoba Kampung Bahari, Nyalakan P...","Rabu, 09 Mar 2022 16:50 WIB\n\n","Polisi mengungkapkan\n\n ""Ada petasan sebagai ...",www.detik.com


In [7]:
column = ['Title', 'Date', 'Description', 'Source']
df_total = pd.DataFrame()

df =pd.read_csv("scrapped_news.csv")
isi=df['description']
df = df.dropna(subset=['description'], axis=0)
for x in range(0, df.shape[0]):
    text = preprocessing (df.iloc[x, -2])
    df.iloc[x, -2] = (' '.join(text)) 

In [8]:
df.head()
df_train, df_test=train_test_split(df,test_size=0.1)
df_train.head()

,title,date,description,source
1170,9 Penadah Motor Hasil Curian di Probolinggo Di...,"Minggu, 23 Jan 2022 08:09 WIB\n\n",sembilan kamis polisi menangkap m hodli mengak...,www.detik.com
182,Polda Metro Ikut Terjun Selidiki Perampokan Rp...,"Rabu, 10 Nov 2021 21:32 WIB\n\n",milik warga pantai indah kapuk pik jakarta uta...,www.detik.com
637,Polisi Periksa 12 Saksi Terkait Penembakan Ust...,"Jumat, 24 Sep 2021 19:44 WIB\n\n",polisi mendalami saksi pemeriksaan ya menganal...,www.detik.com
1426,Ulah Oknum Polisi Peras Wanita Berujung Jadi T...,"Selasa, 22 Des 2020 05:22 WIB\n\n",mis didampingi kuasa hukumnya melaporkan oknum...,www.detik.com
1351,"Vonis 2,5 Tahun Bui untuk Polisi Pemeras PSK d...","Jumat, 04 Jun 2021 22:21 WIB\n\n",wanita berinisial mis tenang korban mis diperg...,www.detik.com


In [9]:
desc_text_train = []
desc_text_test = []

for x in range(0, df_train.shape[0]):
  desc_text_train.append(df_train.iloc[x, -2])

for x in range(0, df_test.shape[0]):
  desc_text_test.append(df_test.iloc[x, -2])

In [10]:
print(df_test.info())
print("Panjang Dokumen Test : ", len(desc_text_test))

print ('-'*90)

print(df_train.info())
print("Panjang Dokumen Train : ", len(desc_text_train))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181 entries, 1457 to 1393
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        181 non-null    object
 1   date         181 non-null    object
 2   description  181 non-null    object
 3   source       181 non-null    object
dtypes: object(4)
memory usage: 7.1+ KB
None
Panjang Dokumen Test :  181
------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1623 entries, 1170 to 830
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1623 non-null   object
 1   date         1623 non-null   object
 2   description  1623 non-null   object
 3   source       1623 non-null   object
dtypes: object(4)
memory usage: 63.4+ KB
None
Panjang Dokumen Train :  1623


In [11]:
# Simpan teks asli dokumen
joblib.dump(desc_text_test, "desc_text_test.pkl")
joblib.dump(desc_text_train, "desc_text_train.pkl")

# Simpan dataframe hasil text processing sebelum TF-IDF Vectorizer
df.to_csv('df_total.csv',index=False )
df_test.to_csv('df_test.csv', index=False)
df_train.to_csv('df_train.csv', index=False)

In [12]:
# Fit transform menggunakan TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_test_matrix = tfidf_vectorizer.fit_transform(desc_text_test)
tfidf_train_matrix = tfidf_vectorizer.fit_transform(desc_text_train)

print('tfidf_test_matrix')
print(tfidf_test_matrix.shape)
print('tfidf_train_matrix')
print(tfidf_train_matrix.shape)


tfidf_test_matrix
(181, 4378)
tfidf_train_matrix
(1623, 14576)


In [13]:
joblib.dump(tfidf_vectorizer, "vectorizer.pkl")
joblib.dump(tfidf_test_matrix, "tfidf_test.pkl")
joblib.dump(tfidf_train_matrix, "tfidf_train.pkl")

['tfidf_train.pkl']

In [14]:
df_total = pd.read_csv('df_total.csv')
df_total = df_total[pd.notnull(df_total['description'])]
print(df_total.info())
print ('-'*90)

document_text_train= joblib.load('desc_text_train.pkl')
document_text_test= joblib.load('desc_text_test.pkl')
print(len(document_text_test))
print(len(document_text_train))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1804 entries, 0 to 1803
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1804 non-null   object
 1   date         1804 non-null   object
 2   description  1804 non-null   object
 3   source       1804 non-null   object
dtypes: object(4)
memory usage: 70.5+ KB
None
------------------------------------------------------------------------------------------
181
1623


In [15]:
kueri='pencurian'
kueri=preprocessing(kueri)
kueri= [" ".join (kueri)]
print (kueri)

tfidf_matrix =joblib.load( "tfidf_train.pkl" )
tfidf_vectorizer = joblib.load( "vectorizer.pkl" ) 
query_vec= tfidf_vectorizer.transform(kueri)
results=cosine_similarity(tfidf_matrix, query_vec).reshape((-1))
#print (results)
print("\n======================\n")
print("Top 10 most similar documents in corpus:")

j = 1

for i in results.argsort()[-10:][::-1]:
    print(j)
    print("No ID Dokumen  : ", i)
    print("Tanggal        : ", df_total.iloc[i,1])
    print("Isi berita     : ", df_total.iloc[i,2])
    print("(Score: %.4f) " % results[i])
    j += 1

['pencurian']


Top 10 most similar documents in corpus:
1
No ID Dokumen  :  492
Tanggal        :  Jumat, 04 Mar 2022 18:14 WIB


Isi berita     :  wanita muda berinisial aw ditemukan tewas busana kamar kos sawah jakarta barat korban diduga diperkosa dibunuh pelaku kapolsek sawah kompol maulana mukarom membenarkan penemuan mayat diduga korban pembunuhan korban ditemukan tewas wib sore hasil olah tkp dugaannya korban dibunuh diperkosa kompol maulana dihubungi peristiwa kawasan mangga selatan sawah jakarta pusat ketua rt kamarut zaman korban kali ditemukan kakaknya wib nggak kronologis posisinya informasi kakaknya pagi korban ditelepon nggak kakak korban kosan korban pas kakaknya kamarut aw diduga korban pemerkosaan korban ditemukan busana diduga korban pemerkosaan posisi korban meninggal menyebut korban meninggal kamar kosnya bantuan ya posisinya si korban
(Score: 0.3856) 
2
No ID Dokumen  :  420
Tanggal        :  Selasa, 22 Feb 2022 01:06 WIB


Isi berita     :  orang pria perempuan di